In [ ]:

! pip install lightgbm catboost optuna -q
import numpy as np
import pandas as pd
import lightgbm as lgb
import catboost as cb
import optuna

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

SEED = 42
N_SPLITS = 5
N_TRIALS = 80
EARLY_STOPPING = 200

TARGET = "Y"
ID_COL = "X1"


train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

y = train[TARGET].copy()
test_ids = test[ID_COL]

train.drop(columns=[TARGET, ID_COL], inplace=True)
test.drop(columns=[ID_COL], inplace=True)


num_cols = train.select_dtypes(include=['int64','float64']).columns
cat_cols = train.select_dtypes(include=['object']).columns

for col in num_cols:
    median = train[col].median()
    train[col].fillna(median, inplace=True)
    test[col].fillna(median, inplace=True)

for col in cat_cols:
    train[col].fillna("Missing", inplace=True)
    test[col].fillna("Missing", inplace=True)


print("Target skew:", y.skew())
use_log = y.skew() > 0.75

if use_log:
    y = np.log1p(y)
    print("Log transform applied")


kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

for col in cat_cols:
    train[col + "_te"] = 0
    test[col + "_te"] = 0

for fold, (tr_idx, val_idx) in enumerate(kf.split(train)):

    X_tr, X_val = train.iloc[tr_idx], train.iloc[val_idx]
    y_tr = y.iloc[tr_idx]

    for col in cat_cols:
        medians = pd.DataFrame({col: X_tr[col], TARGET: y_tr}).groupby(col)[TARGET].median()

        train.loc[val_idx, col + "_te"] = train.loc[val_idx, col].map(medians)
        test[col + "_te"] += test[col].map(medians).fillna(y_tr.median()) / N_SPLITS


train.drop(columns=cat_cols, inplace=True)
test.drop(columns=cat_cols, inplace=True)


def lgb_objective(trial):

    params = {
        "objective": "huber",
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.03, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 40, 120),
        "max_depth": trial.suggest_int("max_depth", 5, 9),
        "min_child_samples": trial.suggest_int("min_child_samples", 30, 150),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 3.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 3.0),
        "n_estimators": 3000,
        "random_state": SEED,
        "n_jobs": -1
    }

    oof = np.zeros(len(train))

    for tr_idx, val_idx in kf.split(train):

        X_tr, X_val = train.iloc[tr_idx], train.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            callbacks=[lgb.early_stopping(EARLY_STOPPING, verbose=False)]
        )

        oof[val_idx] = model.predict(X_val)

    return mean_absolute_error(y, oof)

lgb_study = optuna.create_study(direction="minimize")
lgb_study.optimize(lgb_objective, n_trials=N_TRIALS, show_progress_bar=True)

best_lgb_params = lgb_study.best_params
print("Best LGB MAE:", lgb_study.best_value)

def cat_objective(trial):

    params = {
        "loss_function": "MAE",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.05, log=True),
        "depth": trial.suggest_int("depth", 5, 9),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 5.0),
        "iterations": 3000,
        "random_seed": SEED,
        "early_stopping_rounds": EARLY_STOPPING,
        "verbose": False
    }

    oof = np.zeros(len(train))

    for tr_idx, val_idx in kf.split(train):

        X_tr, X_val = train.iloc[tr_idx], train.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        model = cb.CatBoostRegressor(**params)
        model.fit(X_tr, y_tr, eval_set=(X_val, y_val), use_best_model=True)

        oof[val_idx] = model.predict(X_val)

    return mean_absolute_error(y, oof)

cat_study = optuna.create_study(direction="minimize")
cat_study.optimize(cat_objective, n_trials=N_TRIALS, show_progress_bar=True)

best_cat_params = cat_study.best_params
print("Best CAT MAE:", cat_study.best_value)


oof_lgb = np.zeros(len(train))
oof_cat = np.zeros(len(train))
test_lgb = np.zeros(len(test))
test_cat = np.zeros(len(test))

for fold, (tr_idx, val_idx) in enumerate(kf.split(train)):

    X_tr, X_val = train.iloc[tr_idx], train.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    lgb_model = lgb.LGBMRegressor(**best_lgb_params,
                                  objective="huber",
                                  n_estimators=4000,
                                  random_state=SEED)

    lgb_model.fit(X_tr, y_tr,
                  eval_set=[(X_val, y_val)],
                  callbacks=[lgb.early_stopping(EARLY_STOPPING, verbose=False)])

    oof_lgb[val_idx] = lgb_model.predict(X_val)
    test_lgb += lgb_model.predict(test) / N_SPLITS

    cat_model = cb.CatBoostRegressor(**best_cat_params,
                                     loss_function="MAE",
                                     iterations=4000,
                                     random_seed=SEED,
                                     early_stopping_rounds=EARLY_STOPPING,
                                     verbose=False)

    cat_model.fit(X_tr, y_tr, eval_set=(X_val, y_val), use_best_model=True)

    oof_cat[val_idx] = cat_model.predict(X_val)
    test_cat += cat_model.predict(test) / N_SPLITS


best_mae = 1e9
best_w = 0

for w in np.linspace(0, 1, 101):
    blended = w * oof_lgb + (1 - w) * oof_cat
    mae = mean_absolute_error(y, blended)

    if mae < best_mae:
        best_mae = mae
        best_w = w

print("Best Blend Weight:", best_w)
print("Blended MAE:", best_mae)

final_test = best_w * test_lgb + (1 - best_w) * test_cat

if use_log:
    final_test = np.expm1(final_test)


submission = pd.DataFrame({
    ID_COL: test_ids.values,
    TARGET: final_test
})

submission.reset_index(drop=True, inplace=True)
submission.to_csv("submission.csv", index=True)

print("Submission saved successfully.")

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

[I 2026-02-21 00:50:11,407] A new study created in memory with name: no-name-d9170786-b85e-4e80-bf63-b811a8664bc9


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

  0%|          | 0/80 [00:00<?, ?it/s]

[I 2026-02-21 00:50:14,459] Trial 0 finished with value: 0.3997776399566759 and parameters: {'learning_rate': 0.04488694671803637, 'depth': 5, 'l2_leaf_reg': 1.7316578252945773}. Best is trial 0 with value: 0.3997776399566759.
[I 2026-02-21 00:50:18,379] Trial 1 finished with value: 0.40044034193266603 and parameters: {'learning_rate': 0.03823593705540547, 'depth': 6, 'l2_leaf_reg': 3.0703651266570877}. Best is trial 0 with value: 0.3997776399566759.
[I 2026-02-21 00:50:26,132] Trial 2 finished with value: 0.4003445906034423 and parameters: {'learning_rate': 0.022302783357574796, 'depth': 7, 'l2_leaf_reg': 4.814135924999626}. Best is trial 0 with value: 0.3997776399566759.
[I 2026-02-21 00:50:39,112] Trial 3 finished with value: 0.4037467644668069 and parameters: {'learning_rate': 0.029375680982242598, 'depth': 9, 'l2_leaf_reg': 2.484263599857385}. Best is trial 0 with value: 0.3997776399566759.
[I 2026-02-21 00:50:46,715] Trial 4 finished with value: 0.3993773113827421 and parameters: